In [ ]:
# Ref: https://dev.freeagent.com/docs
# 1. Generate a token using OAuth playground on this link https://dev.freeagent.com/docs/quick_start
# 2. Enter the client id, secret, authcode, access token in the .env file in this folder

import os, json
import requests
import pandas as pd
import urllib.request
from pathlib import Path
from pdfminer.high_level import extract_text as pdf_extract_text

%reload_ext dotenv
%dotenv
client_id = os.getenv('FREEAG_OAUTH_CLIENT_ID', None)
client_secret = os.getenv('FREEAG_OAUTH_CLIENT_SECRET', None)
auth_code = os.getenv('FREEAG_AUTH_CODE', None)
access_token = os.getenv('FREEAG_ACCESS_TOKEN', None)
account_map = json.loads(os.getenv('ACCOUNTS', None).replace("\\", ""))


In [ ]:

from_date = '2022-05-10'
#url = f'https://api.freeagent.com/v2/bills?updated_since={time_filter}'
url = f'https://api.freeagent.com/v2/bills?from_date={from_date}'

headers = {
    'Content-Type': 'application/json',
    'Charset': 'UTF-8',
    'Authorization': f'Bearer {access_token}'
}
bill_entities = ['Supplier: Amazon Web Services', 'Supplier: OVHcloud', 
'Supplier: Snyk Ltd', '']
#use the 'auth' parameter to send requests with HTTP Basic Auth:
#x = requests.post(url, data = myobj, auth = (client_id, client_secret))
bills = requests.get(url, headers=headers)
print(bills.reason)
#print(bills.json()['bills'][2])

In [ ]:
df_bills = pd.DataFrame(columns=['comments', 'paid_value'])
search_keywords = ['Account number:']
invoice_json = {
    "invoice": {
        "contact": "xxx",
        "dated_on": "2022-14-04",
        "payment_terms_in_days": 7,
        "invoice_items" : []
    }
}
for bill in bills.json()['bills']:
    if(bill['comments'] in bill_entities):
        r = requests.get(bill['attachment']['content_src'])
        print(f'{1} {2} {3}', r.status_code, r.reason, r)
        filename = bill['attachment']['file_name']
        file_obj = Path(filename)
        file_obj.write_bytes(r.content)

        if filename.endswith('.pdf'):
            print(filename)
            try:
                text = pdf_extract_text(filename)
                sentences = text.split("\n")
                account = ''
                for sentence in sentences:
                    for word in account_map.keys():
                        if word in sentence:
                            account = word
                            break
                    if account:
                        break
                print(account)
            except Exception as e:
                print(e)
        

        df_bills = df_bills.append({
           'comments': bill['comments'],
           'reference': bill['reference'],
           'paid_value': bill['paid_value'],
           'total_value': bill['total_value'],
           'sales_tax_value': bill['sales_tax_value'],
           'net_value': bill['net_value'],
           'attachment': bill['attachment']['file_name'],
           'dated_on': bill['dated_on'],
           'paid_on': bill['paid_on'] if 'paid_on' in bill else '',
           'due_on': bill['due_on'],
           #'attachment_full': bill['attachment'],
           'attachmentd': bill['attachment']['url'],
           'ref_account1': account,
           'ref_account2': '' if not account else account_map[account],
            }, ignore_index=True
        )
        invoice_json["invoice"]["invoice_items"].append({
            "project": "XXXXX",
            "description":  bill['comments'],
            "item_type": "Bills",
            "price": float(bill['total_value']) - float(bill['sales_tax_value']),
            "sales_tax_rate": "20",
            "quantity": 0,
        })

print(df_bills)
#df_bills.iloc[0]['attachment_full']
df_bills.to_excel('output.xlsx')

In [ ]:
json.dumps(invoice_json, indent=4)
invoice_id = 3123123123
url = f'https://api.freeagent.com/v2/invoices/:{invoice_id}'
headers = {
    'Content-Type': 'application/json',
    'Charset': 'UTF-8',
    'Authorization': f'Bearer {access_token}'
}
#use the 'auth' parameter to send requests with HTTP Basic Auth:
#x = requests.post(url, data = myobj, auth = (client_id, client_secret))
invoices_resp = requests.post(url, headers=headers, data = invoice_json)
print(invoices_resp)
print(invoices_resp.reason)

In [ ]:
invoice_id = 48377900
url = f'https://api.freeagent.com/v2/invoices/:{invoice_id}'
headers = {
    'Content-Type': 'application/json',
    'Charset': 'UTF-8',
    'Authorization': f'Bearer {access_token}'
}
#use the 'auth' parameter to send requests with HTTP Basic Auth:
#x = requests.post(url, data = myobj, auth = (client_id, client_secret))
invoices_resp = requests.post(url, headers=headers, data = invoice_json)
print(invoices_resp)
print(invoices_resp.reason)

In [ ]:
df_bills = pd.DataFrame.from_dict(bills.json()['bills'])
df_bills[['dated_on', 'due_on', 'comments', 'reference', 'total_value', 
    'net_value','sales_tax_value','paid_value', 'due_value', 'status', 
    'attachment']]
    
#refresh token generation
url = 'https://api.freeagent.com/v2/token_endpoint'
headers = {
    'Content-Type': 'application/json',
    'Charset': 'UTF-8'
}
myobj = {'grant_type':'authorization_code',
 'code':'1Hd9Dkno705CwdWhzma-fIGQIbfamdXfYPmCM0xiu',
 'client_id': client_id,
 'client_secret': client_secret,
 'redirect_uri':'https://developers.google.com/oauthplayground'
 }

#use the 'auth' parameter to send requests with HTTP Basic Auth:
#x = requests.post(url, data = myobj, auth = (client_id, client_secret))
x = requests.post(url, headers=headers, data = myobj)
print(x.status_code)
print(x.reason)


In [ ]:

time_filter = '2021-12-31T00:00:00.000Z'

url = 'https://api.freeagent.com/v2/invoices?sort=-updated_at&?updated_since={time}'
headers = {
    'Content-Type': 'application/json',
    'Charset': 'UTF-8',
    'Authorization': f'Bearer {access_token}'
}
#use the 'auth' parameter to send requests with HTTP Basic Auth:
#x = requests.post(url, data = myobj, auth = (client_id, client_secret))
invoices = requests.get(url, headers=headers)
df_invoices = pd.DataFrame.from_dict(invoices.json()['invoices'])
df_invoices[['dated_on', 'due_on', 'reference', 'total_value', 'paid_value', 'exchange_rate', 'due_value', 'status']]